# 🏟️ Soccer Banner Segmentation - Training Pipeline

Questo notebook implementa il training del modello YOLOv11 per la segmentazione dei banner.

## Prerequisiti
**⚠️ Prima di eseguire questo notebook:**
1. Eseguire `SBS EDA.ipynb` per preparare i dataset
2. Verificare che le cartelle `dataset/kaggle_dataset` e `dataset/roboflow_dataset` esistano

## Architettura del Training
Il training segue un approccio **Two-Stage Transfer Learning**:
1. **Fase A (Expert)**: Addestramento su dataset Kaggle (ampio, generico)
2. **Fase B (Specialist)**: Fine-tuning su dataset Roboflow (specifico, alta qualità)

## Versionamento Automatico
I modelli vengono salvati nella cartella `model/` con versioning automatico (v1, v2, v3...).

---

## 📚 1. Import delle Librerie

In [ ]:
# ============================================================================
# IMPORT LIBRERIE
# ============================================================================

import os
import re
import shutil
from pathlib import Path

import torch
from ultralytics import YOLO  # type: ignore

print("✅ Ambiente configurato.")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 📁 2. Configurazione Directory

In [ ]:
# ============================================================================
# CONFIGURAZIONE DIRECTORY (create da SBS EDA.ipynb)
# ============================================================================

BASE_DIR = Path(os.getcwd())
DATASETS_DIR = BASE_DIR / "dataset"
OUTPUT_DIR = BASE_DIR / "output"
MODEL_DIR = BASE_DIR / "model"

# Path ai file YAML dei dataset
KAG_YAML_PATH = str(DATASETS_DIR / 'kaggle_dataset' / 'kaggle_data.yaml')
ROBO_YAML_PATH = str(DATASETS_DIR / 'roboflow_dataset' / 'data.yaml')

# Verifica esistenza dataset
if not os.path.exists(KAG_YAML_PATH):
    raise FileNotFoundError(f"❌ Dataset Kaggle non trovato. Esegui prima SBS EDA.ipynb")
if not os.path.exists(ROBO_YAML_PATH):
    raise FileNotFoundError(f"❌ Dataset Roboflow non trovato. Esegui prima SBS EDA.ipynb")

print("✅ Directory e dataset trovati:")
print(f"   📂 Kaggle:   {KAG_YAML_PATH}")
print(f"   📂 Roboflow: {ROBO_YAML_PATH}")
print(f"   📂 Output:   {OUTPUT_DIR}")
print(f"   📂 Model:    {MODEL_DIR}")

## 🔢 3. Funzione di Versioning Automatico

In [ ]:
# ============================================================================
# VERSIONING AUTOMATICO DEI MODELLI
# ============================================================================

def get_next_version(model_dir: Path, model_name: str) -> int:
    """
    Trova la prossima versione disponibile per un modello.
    Cerca cartelle esistenti con pattern: model_name_v1, model_name_v2, etc.
    
    Returns:
        Prossimo numero di versione da usare
    """
    existing_versions = []
    
    if model_dir.exists():
        for item in model_dir.iterdir():
            if item.is_dir() and item.name.startswith(model_name):
                # Cerca pattern _v{numero}
                match = re.search(r'_v(\d+)$', item.name)
                if match:
                    existing_versions.append(int(match.group(1)))
                elif item.name == model_name:
                    # Se esiste senza versione, consideralo v0
                    existing_versions.append(0)
    
    if not existing_versions:
        return 1
    
    return max(existing_versions) + 1


def save_model_with_version(source_weights: Path, model_dir: Path, model_name: str) -> Path | None:
    """
    Salva i pesi del modello nella cartella model/ con versioning automatico.
    
    Args:
        source_weights: Path ai pesi best.pt generati dal training
        model_dir: Directory dove salvare i modelli (model/)
        model_name: Nome base del modello (es. 'expert_model', 'specialist_model')
    
    Returns:
        Path alla cartella del modello salvato
    """
    if not source_weights.exists():
        print(f"❌ Pesi non trovati: {source_weights}")
        return None
    
    # Crea directory model/ se non esiste
    model_dir.mkdir(parents=True, exist_ok=True)
    
    # Determina la versione
    version = get_next_version(model_dir, model_name)
    versioned_name = f"{model_name}_v{version}"
    
    # Crea cartella per questa versione
    version_dir = model_dir / versioned_name
    version_dir.mkdir(parents=True, exist_ok=True)
    
    # Copia i pesi
    dest_weights = version_dir / 'best.pt'
    shutil.copy(source_weights, dest_weights)
    
    print(f"   📦 Modello salvato: {versioned_name}")
    print(f"   📁 Path: {dest_weights}")
    
    return version_dir


print("✅ Funzioni di versioning caricate.")

## 🏋️ 4. Fase A: Training Expert Model (Kaggle)

Primo stage di training sul dataset Kaggle per apprendere le feature generali dei banner.

In [ ]:
# ============================================================================
# FASE A: TRAINING EXPERT MODEL
# ============================================================================

def run_expert_training(kag_yaml: str):
    """
    Fase A: addestramento Expert su dataset Kaggle.
    I pesi vengono salvati con versioning automatico.
    """
    print("="*60)
    print("🏋️ FASE A: Training Expert Model")
    print("   Dataset: Kaggle (1600+ immagini)")
    print("="*60)
    
    model = YOLO('yolo11n-seg.pt')
    device = 0 if torch.cuda.is_available() else 'cpu'
    print(f"\n⚙️  Dispositivo: {'GPU (CUDA)' if device == 0 else 'CPU'}")
    
    results = model.train(
        data=kag_yaml,
        epochs=50,
        imgsz=640,
        batch=16,
        project=str(OUTPUT_DIR),
        name='expert_model',
        exist_ok=True,
        verbose=True,
        device=device
    )
    
    best_weights = OUTPUT_DIR / 'expert_model' / 'weights' / 'best.pt'
    
    # Salva modello con versioning automatico
    saved_dir = save_model_with_version(best_weights, MODEL_DIR, 'expert_model')
    
    print("\n" + "="*60)
    print("✅ FASE A COMPLETATA")
    print(f"   📦 Training output: {OUTPUT_DIR / 'expert_model'}")
    if saved_dir:
        print(f"   📦 Modello versionato: {saved_dir}")
    print("="*60)
    
    return saved_dir

# Avvio Fase A
expert_model_dir = run_expert_training(KAG_YAML_PATH)

## 🎯 5. Fase B: Fine-Tuning Specialist Model (Roboflow)

Secondo stage: specializzazione del modello Expert sul dataset Roboflow.

In [ ]:
# ============================================================================
# FASE B: FINE-TUNING SPECIALIST MODEL (OTTIMIZZATA VELOCITÀ/QUALITÀ)
# ============================================================================

def run_specialist_fine_tuning(robo_yaml: str):
    """
    Fase B: fine-tuning su dataset Roboflow v4.
    Ottimizzato con sblocco backbone e alta risoluzione per gestire occlusioni (rete).
    """
    print("="*60)
    print("🎯 FASE B: Fine-Tuning Specialist Model")
    print("   Dataset: Roboflow v4 (99 train, 9 valid)")
    print("="*60)
    
    expert_weights = OUTPUT_DIR / 'expert_model' / 'weights' / 'best.pt'
    
    if not expert_weights.exists():
        raise FileNotFoundError(
            f"❌ Pesi Expert non trovati: {expert_weights}\n"
            f"   Esegui prima la Fase A."
        )
    
    print(f"\n📦 Caricamento pesi Expert: {expert_weights.name}")
    model = YOLO(str(expert_weights))
    
    print("\n⚙️  Configurazione training Ultra High Quality (Unfrozen + High Res)...")
    
    model.train(
        data=robo_yaml,
        epochs=130,            # AUMENTATO: Più tempo per adattamento fine
        imgsz=1024,            # AUMENTATO: Alta risoluzione per distinguere rete da banner
        batch=6,               # RIDOTTO: Per gestire 1024 + unfreeze
        
        # Fine-Tuning Avanzato
        freeze=0,              # SBLOCCATO: Fondamentale per imparare la texture della rete
        lr0=0.0005, 
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        dropout=0.1,           # AGGIUNTO: Regularization per prevenire overfitting
        
        # Maschere & Qualità
        retina_masks=True,
        overlap_mask=False,
        
        # Augmentation (Occlusion Robustness)
        degrees=15.0,
        perspective=0.0005,
        translate=0.2,
        scale=0.8,             # AUMENTATO
        shear=0.0,
        mosaic=1.0,
        mixup=0.15,
        copy_paste=0.3,
        erasing=0.3,
        hsv_h=0.02,
        hsv_s=0.6,
        hsv_v=0.6,
        
        # Meccaniche
        multi_scale=True,
        patience=30,
        close_mosaic=10,
        
        project=str(OUTPUT_DIR),
        name='specialist_model',
        exist_ok=True,
        verbose=True,
        plots=True,
        save_period=-1,
        device=0 if torch.cuda.is_available() else 'cpu',
        workers=4,
        amp=True
    )
    
    final_weights = OUTPUT_DIR / 'specialist_model' / 'weights' / 'best.pt'
    
    # Salva modello con versioning automatico
    saved_dir = save_model_with_version(final_weights, MODEL_DIR, 'specialist_model')
    
    print("\n" + "="*60)
    print("🎉 TRAINING COMPLETATO")
    print("="*60)
    print(f"\n📦 Training output: {OUTPUT_DIR / 'specialist_model'}")
    if saved_dir:
        print(f"📦 Modello versionato: {saved_dir}")
    print("="*60)
    
    return saved_dir

# Avvio Fase B
specialist_model_dir = run_specialist_fine_tuning(ROBO_YAML_PATH)
